<a href="https://colab.research.google.com/github/MohsenJadidi/Erdos-Goes-Neural/blob/main/ModifiedCode/Cut_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_scatter
!pip install torch_sparse
!pip install torch_geometric


In [2]:
!apt-get install python3-dev graphviz libgraphviz-dev pkg-config
!pip install graphviz
!pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [3]:
!pip install visdom
!pip install GPUtil

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('/content/drive/My Drive/ISLR Course - Fall 2020/Project')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import os
import pickle
import torch
import torch.nn.functional as F
from torch.nn import Linear
# from kernel.datasets import get_dataset
import time
from torch import tensor
from torch.optim import Adam
from torch.optim import SGD
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from math import ceil
from torch.nn import Linear
from torch.distributions import categorical
from torch.distributions import Bernoulli
import torch.nn
%matplotlib inline
from matplotlib import pyplot as plt
import pygraphviz as pgv
from torch_geometric.utils import convert as cnv
from torch_geometric.utils import sparse as sp
from torch_geometric.data import Data
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
from torch.utils.data.sampler import RandomSampler
from torch.nn.functional import gumbel_softmax
from torch.distributions import relaxed_categorical
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.inits import glorot, zeros
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
from torch_geometric.nn import GINConv, GATConv
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
# from torch_geometric.utils import scatter_
from torch_geometric.data import Batch 
from torch_scatter import scatter_min, scatter_max, scatter_add, scatter_mean
from torch import autograd
from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops
from models import cut_MPNN
from modules_and_utils import derandomize_cut, GATAConv,get_diracs
import scipy
import scipy.io
import GPUtil

In [ ]:
datasets = ["facebook", "sf","twitter"]
curr_dataset= datasets[0] 
#set random seed
rseed = 201

if curr_dataset=="facebook":
    datasetname = "facebook_graphs"
    dataset = []
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #set up facebook data 
    upper_limit = 15000
    lower_limit = 0
    path_to_facebook_dataset= "%PATH"
        for file in os.listdir(path_to_facebook_dataset):
            if file.endswith(".mat"):
                adj_matrix = scipy.io.loadmat(path_to_facebook_dataset+str(file))
                edge_index = from_scipy_sparse_matrix(adj_matrix['A'])[0]
                x = torch.ones(adj_matrix['local_info'].shape[0])
                if (adj_matrix['local_info'].shape[0] < lower_limit) or (adj_matrix['local_info'].shape[0] > upper_limit):
                    continue
                data_temp = Batch(x = x, edge_index = edge_index.long(), batch = torch.zeros_like(x).long())
                data_proper = get_diracs(data_temp.to('cuda'), 1, sparse = True)
                r,c = data_proper.edge_index
                data = Batch(x = data_temp.x, edge_index = data_temp.edge_index)
                degrees = degree(r, adj_matrix['local_info'].shape[0])
                print("Graph specs: ")
                print("number of nodes: ", adj_matrix['A'].shape[0])
                print("average degree: ", degrees.mean(0))
                print("total volume: ", data_proper.total_vol)
                print("-------------")
                dataset += [data]

else if curr_dataset=="sf":
    datasetname = "SF-295"
    dataset = get_dataset(datasetname,sparse=1)

else if curr_dataset=="twitter"
    path_to_twitter_dataset = "%PATH"
    stored_dataset = open(path_to_twitter_dataset, 'rb')        
    dataset = pickle.load(stored_dataset)    

dataset_scale = 1.
total_samples = int(np.floor(len(dataset)*dataset_scale))
dataset = dataset[:total_samples]

num_trainpoints = int(np.floor(0.6*len(dataset)))
num_valpoints = int(np.floor(num_trainpoints/3))
num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)


traindata= dataset[0:num_trainpoints]
valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]

testdata = dataset[num_trainpoints + num_valpoints:]

batch_size = 32
train_loader = DataLoader(traindata, batch_size, shuffle=True)
test_loader = DataLoader(testdata, batch_size, shuffle=False)
val_loader =  DataLoader(valdata, batch_size, shuffle=False)


#set up random seed 
torch.manual_seed(rseed)
np.random.seed(2)   
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Another dataset

In [7]:
from torch_geometric.datasets import TUDataset
import numpy as np

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset_name = "ENZYMES"

dataset_scale = 1
total_samples = int(np.floor(len(dataset)*dataset_scale))
dataset = dataset[:total_samples]

num_trainpoints = int(np.floor(0.6*len(dataset)))
num_valpoints = int(np.floor(num_trainpoints/3))
num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)

traindata= dataset[0:num_trainpoints]
valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]
testdata = dataset[num_trainpoints + num_valpoints:]

batch_size = 32

train_loader = DataLoader(traindata, batch_size, shuffle=True)
test_loader = DataLoader(testdata, batch_size, shuffle=False)
val_loader =  DataLoader(valdata, batch_size, shuffle=False)


#set up random seeds 
torch.manual_seed(1)
np.random.seed(2)   
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Train

In [8]:
def predict(model, data_loader, recfield):
    net.eval()
    avg_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.to(device)
        data = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = recfield)
        data = data.to(device)
        retdict = net(data)
        avg_loss += retdict['loss'][0].item()/len(data_loader)

    return avg_loss

In [10]:
epochs=150
numlayers=6
elasticity = 0.25
receptive_field= numlayers + 1
val_losses = []

#for sf/twitter
#net =  cut_MPNN(dataset,numlayers, 64, 64,1, elasticity = elasticity)

#for faceboook
net =  cut_MPNN(dataset,6, 64, 1,1, elasticity = 0.25)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lr_decay_step_size = 5
lr_decay_factor = 0.95

net.to(device).reset_parameters()
optimizer = Adam(net.parameters(), lr=0.0001, weight_decay=0.00)
net.train()
retdict = {}
path_to_save = "%PATH"
for epoch in range(epochs):
    totalretdict = {}
    count=0

#-----------------------------------
    # if epoch % 15 == 0:
    #     val_losses += [predict(net, val_loader, receptive_field)]
    #     if epoch>20:
    #         if (val_losses[-1] > val_losses[-2]) and (val_losses[-1] > val_losses[-3]) and (epoch>100):
    #             print("Converged!")
    #             if val_losses[-2] < val_losses[-3]:
    #                 file_name = path_to_save+ str(datasetname)+ '/' + str(net)+ '_'+ str(epoch-15)+'_samples2'
    #                 net.load_state_dict(torch.load(file_name), strict=False)
    #             else:
    #                 file_name = path_to_save+ str(datasetname)+ '/' + str(net)+ '_'+ str(epoch-30)+'_samples2'
    #                 net.load_state_dict(torch.load(file_name), strict=False)
    #             break
    #         file_name = path_to_save + str(datasetname)+ '/' + str(net)+ '_'+ str(epoch)+'_samples2'
    #         print("file_name: ", file_name)
    #         torch.save({file_name : net.state_dict()},  file_name)
    
    #learning rate schedule
    if epoch % lr_decay_step_size == 0:
        for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_decay_factor * param_group['lr']

    #show currrent epoch and GPU utilizationss
    print('Epoch: ', epoch)
    GPUtil.showUtilization()

    net.train()

    for data in train_loader:
        count += 1 
        optimizer.zero_grad(), 
        data = data.to(device)
        data_prime = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = receptive_field)

        data = data.to('cpu')
        data_prime = data_prime.to(device)  

        retdict = net(data_prime)

        for key,val in retdict.items():
            if "sequence" in val[1]:
                if key in totalretdict:
                    totalretdict[key][0] += val[0].item()
                else:
                    totalretdict[key] = [val[0].item(),val[1]]
        
        if epoch > 0:
                retdict["loss"][0].backward()
                torch.nn.utils.clip_grad_norm_(net.parameters(),1)
                optimizer.step()                   

        del data_prime

Epoch:  0
| ID | GPU | MEM |
------------------
|  0 |  0% |  7% |
111111111
22222222


IndexError: ignored